In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Crop_Price_India.csv")
df.head()

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Andaman and Nicobar,South Andaman,Port Blair,Amaranthus,Amaranthus,04/03/2019,6000,8000,7000
1,Andaman and Nicobar,South Andaman,Port Blair,Banana - Green,Banana - Green,04/03/2019,4500,5500,5000
2,Andaman and Nicobar,South Andaman,Port Blair,Bhindi(Ladies Finger),Bhindi,04/03/2019,6000,8000,7000
3,Andaman and Nicobar,South Andaman,Port Blair,Bitter gourd,Other,04/03/2019,6000,8000,7000
4,Andaman and Nicobar,South Andaman,Port Blair,Black pepper,Other,04/03/2019,110000,130000,120000


In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['state'] = label_encoder.fit_transform(df['state'])
df['district'] = label_encoder.fit_transform(df['district'])
df['market'] = label_encoder.fit_transform(df['market'])
df['commodity'] = label_encoder.fit_transform(df['commodity'])
df['variety'] = label_encoder.fit_transform(df['variety'])
df = df.drop('arrival_date', axis=1)

In [4]:
df.head()

,state,district,market,commodity,variety,min_price,max_price,modal_price
0,0,153,246,0,17,6000,8000,7000
1,0,153,246,10,30,4500,5500,5000
2,0,153,246,18,39,6000,8000,7000
3,0,153,246,19,148,6000,8000,7000
4,0,153,246,22,148,110000,130000,120000


In [5]:
X = df.drop(columns=['modal_price'])
y = df['modal_price']


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:

svr_rbf = SVR(kernel='rbf')
svr_poly = SVR(kernel='poly')

svr_rbf.fit(X_train, y_train)
svr_poly.fit(X_train, y_train)

SVR(kernel='poly')

In [9]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))  # Adjust input shape
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [10]:
X_train_rnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_rnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, verbose=0)

In [ ]:
svr_rbf_pred = svr_rbf.predict(X_test)
svr_poly_pred = svr_poly.predict(X_test)
rnn_pred = model.predict(X_test_rnn)

In [ ]:
svr_rbf_mse = mean_squared_error(y_test, svr_rbf_pred)
svr_poly_mse = mean_squared_error(y_test, svr_poly_pred)
rnn_mse = mean_squared_error(y_test, rnn_pred)
print("SVR RBF MSE:", svr_rbf_mse)
print("SVR Polynomial MSE:", svr_poly_mse)
print("RNN MSE:", rnn_mse)